In [1]:
import sys
import torch
import tntorch as tn
import numpy as np
sys.path.append('..')
import lib

In [2]:
rho, train_X, train_y = lib.read_data('../data/2019_12_29_haar_dim4_0.dat')
dim = rho.shape[0]
n_qubits = np.log2(dim)

tensor_rank = 2**n_qubits
batch_size = 100
lr = 1e-2

sigma = lib.simulator.randomMixedState(rho.shape[0])
sigma_real, sigma_imag = [tn.Tensor(x, ranks_tt=tensor_rank, requires_grad=True) for x in [np.real(sigma), np.imag(sigma)]]

epoches = 10

In [3]:
def trace(tensor):
    if len(tensor.shape) == 2:
        return sum([tensor[i,i] for i in range(tensor.shape[0])])
    if len(tensor.shape) == 3:
        return sum([tensor[:, i,i] for i in range(tensor.shape[1])])

def cholesky(sigma_real, sigma_imag):
    return sigma_real.dot(sigma_real, k=1)+sigma_imag.dot(sigma_imag, k=1), sigma_real.dot(sigma_imag, k=1)-sigma_imag.dot(sigma_real, k=1)
    
#initial_trace = sum((np.real(trace(test_X.dot(sigma)))-test_y)**2)

def loss(sigma_real, sigma_imag):
    sigma_real, sigma_imag = cholesky(sigma_real, sigma_imag)
    res = 0
    idx = np.random.choice(np.arange(train_X.shape[0]), batch_size)
    for E_m,y_m in zip(train_X[idx], train_y[idx].astype('float64')):
        E_real, E_imag = [tn.Tensor(x) for x in [np.real(E_m), np.imag(E_m)]]
        res += ((E_real.dot(sigma_real)+E_imag.dot(sigma_imag)-y_m)**2)
#     return res/(initial_trace*train_X.shape[0])
    return res
    

def eval_loss(sigma_real, sigma_imag): # any score function can be used here
    sigma_real_, sigma_imag_ = cholesky(sigma_real, sigma_imag)
    sigma = sigma_real_.torch().detach().cpu().numpy() + 1j*sigma_imag_.torch().detach().cpu().numpy()
    return -lib.fidelity(sigma, rho)

#print('Trace before: %f'%initial_trace)

lib.tn_optimize([sigma_real, sigma_imag], loss, eval_loss, tol=0, patience=2000,print_freq=10,lr=lr)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
iter: 0 |  loss: 2.7836837768554688 | eval_loss: -0.12138022681802281 | total time: 0.05395245552062988


/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


iter: 10 |  loss: 0.5683777928352356 | eval_loss: -0.05330325027391999 | total time: 0.7008352279663086
iter: 20 |  loss: 0.6447154879570007 | eval_loss: -0.05134491783655765 | total time: 1.3370280265808105
iter: 30 |  loss: 0.6271519660949707 | eval_loss: -0.04886497952857281 | total time: 1.9662182331085205
iter: 40 |  loss: 0.46272042393684387 | eval_loss: -0.043744643652314355 | total time: 2.6096370220184326
iter: 50 |  loss: 0.39473339915275574 | eval_loss: -0.039872786250100496 | total time: 3.2554726600646973
iter: 60 |  loss: 0.33345332741737366 | eval_loss: -0.0367591776440366 | total time: 3.8809754848480225
iter: 70 |  loss: 0.24817092716693878 | eval_loss: -0.03150163703932454 | total time: 4.500956058502197
iter: 80 |  loss: 0.18692272901535034 | eval_loss: -0.028733818924693927 | total time: 5.130442142486572
iter: 90 |  loss: 0.16947132349014282 | eval_loss: -0.025983265281235495 | total time: 5.775091171264648
iter: 100 |  loss: 0.16792812943458557 | eval_loss: -0.021

iter: 770 |  loss: 8.620744483778253e-05 | eval_loss: -0.0009483014322858434 | total time: 48.515114068984985
iter: 780 |  loss: 8.084448199952021e-05 | eval_loss: -0.000939124045109707 | total time: 49.14062976837158
iter: 790 |  loss: 7.698864646954462e-05 | eval_loss: -0.0009282323592218217 | total time: 49.76325726509094
iter: 800 |  loss: 7.57541783968918e-05 | eval_loss: -0.0009214735781546193 | total time: 50.390586614608765
iter: 810 |  loss: 8.011559839360416e-05 | eval_loss: -0.0009137049095102767 | total time: 51.01661992073059
iter: 820 |  loss: 6.571945414179936e-05 | eval_loss: -0.0009016997279205488 | total time: 51.64347720146179
iter: 830 |  loss: 6.0002566897310317e-05 | eval_loss: -0.000894856044506129 | total time: 52.27366876602173
iter: 840 |  loss: 6.155385926831514e-05 | eval_loss: -0.000887684076854242 | total time: 52.90037441253662
iter: 850 |  loss: 5.647766010952182e-05 | eval_loss: -0.0008752243557212226 | total time: 53.545355796813965
iter: 860 |  loss: 

iter: 1520 |  loss: 1.0176358046010137e-05 | eval_loss: -0.0004912591056374416 | total time: 95.58223628997803
iter: 1530 |  loss: 1.0082917469844688e-05 | eval_loss: -0.000487715868210372 | total time: 96.22974157333374
iter: 1540 |  loss: 8.799485840427224e-06 | eval_loss: -0.0004827809984397479 | total time: 96.85419392585754
iter: 1550 |  loss: 1.0620334251143504e-05 | eval_loss: -0.00047943875806907035 | total time: 97.47981023788452
iter: 1560 |  loss: 8.788882951193955e-06 | eval_loss: -0.00047635957591125866 | total time: 98.09826159477234
iter: 1570 |  loss: 8.498655006405897e-06 | eval_loss: -0.0004706671503399684 | total time: 98.72775101661682
iter: 1580 |  loss: 8.55518283060519e-06 | eval_loss: -0.00046765693914655257 | total time: 99.34366798400879
iter: 1590 |  loss: 9.270617738366127e-06 | eval_loss: -0.0004657485279761784 | total time: 99.96742033958435
iter: 1600 |  loss: 9.284472071158234e-06 | eval_loss: -0.0004618459273959669 | total time: 100.58789896965027
iter: